In [112]:
import torch

In [113]:
input = torch.randn((32, 10, 64,64,64))

In [114]:
target = torch.empty((32,10,64,64,64)).random_(2)

In [115]:
mask = torch.empty((32,10,64,64,64)).random_(2)

In [123]:
class_weights = torch.empty((10,1,1,1)).random_(1,15)

In [137]:
bce = torch.nn.functional.binary_cross_entropy_with_logits(input, target,mask, pos_weight=class_weights, reduction="none")

In [139]:
torch.sum(bce*mask)/torch.sum(mask)

tensor(3.7069)

In [140]:
bce = torch.nn.functional.binary_cross_entropy_with_logits(input, target,mask, pos_weight=class_weights, reduction="sum")
bce/torch.sum(mask)

tensor(3.7069)

In [135]:
bce

tensor([[[[[3.2039e-07, 1.2054e-07, 1.1097e-07,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 2.2929e-08, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.6037e-07,
            0.0000e+00, 0.0000e+00],
           ...,
           [0.0000e+00, 0.0000e+00, 4.7772e-08,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [1.8913e-07, 1.5364e-08, 3.4886e-07,  ..., 1.0078e-07,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 8.3965e-08,  ..., 0.0000e+00,
            2.0775e-07, 1.5872e-07]],

          [[7.7845e-08, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 2.4659e-07],
           [0.0000e+00, 1.0157e-07, 1.7305e-08,  ..., 1.8109e-08,
            0.0000e+00, 6.5564e-08],
           [1.6673e-07, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            2.5105e-08, 2.7258e-07],
           ...,
           [1.7096e-08, 1.8966e-08, 0.0

In [118]:
torch.sum(bce*mask)/torch.sum(mask)

tensor(1.4302)

In [119]:
torch.sum(bce*mask)/(32*10*64*64*64)

tensor(0.7151)

In [120]:
bce_unmasked = torch.nn.functional.binary_cross_entropy_with_logits(input, target, reduction="none")
bce_masked = torch.nn.functional.binary_cross_entropy_with_logits(input, target, mask, reduction="sum")

In [126]:
bcesum = 0
for ch in range(10):
    img = input[:,ch,...].flatten()
    tgt = target[:,ch,...].flatten()
    cw = class_weights[ch,...].flatten()
    msk = mask[:,ch,...].flatten()
    # print(img.shape, tgt.shape, cw.shape)
    # print(int(cw.numpy()))
    bce = torch.nn.functional.binary_cross_entropy_with_logits(img, tgt, msk, pos_weight=cw, reduction='none')
    bce_masked = bce * msk
    print(ch, torch.sum(bce_masked))
    bcesum += torch.sum(bce_masked)
print(int(bcesum.numpy()))
print(bcesum/torch.sum(mask))
    

0 tensor(16900388.)
1 tensor(20300446.)
2 tensor(5070061.)
3 tensor(23660216.)
4 tensor(13512483.)
5 tensor(3380084.5000)
6 tensor(18590090.)
7 tensor(10134759.)
8 tensor(23647880.)
9 tensor(20276116.)
155472528
tensor(3.7069)


In [130]:
bcesum = 0
for ch in range(10):
    img = torch.nn.functional.sigmoid(input[:,ch,...].flatten())
    tgt = target[:,ch,...].flatten()
    cw = class_weights[ch,...].flatten()
    msk = mask[:,ch,...].flatten()
    # print(img.shape, tgt.shape, cw.shape)
    # print(int(cw.numpy()))
    
    bce = -  (cw * tgt * torch.log(img) + (torch.ones_like(tgt)-tgt)*torch.log(torch.ones_like(img)-img))
    #bce = torch.nn.functional.binary_cross_entropy(torch.nn.functional.sigmoid(img), tgt, msk, reduction='none')
    bce_masked = bce * msk
    print(ch, cw, torch.sum(bce_masked))
    bcesum += torch.sum(bce_masked)
print(int(bcesum.numpy()))
print(bcesum / torch.sum(mask))

0 tensor([9.]) tensor(16900388.)
1 tensor([11.]) tensor(20300446.)
2 tensor([2.]) tensor(5070061.)
3 tensor([13.]) tensor(23660220.)
4 tensor([7.]) tensor(13512483.)
5 tensor([1.]) tensor(3380084.5000)
6 tensor([10.]) tensor(18590090.)
7 tensor([5.]) tensor(10134759.)
8 tensor([13.]) tensor(23647880.)
9 tensor([11.]) tensor(20276116.)
155472528
tensor(3.7069)


In [77]:
int(bcesum.numpy())

140291776

In [86]:
int(torch.sum(torch.nn.functional.binary_cross_entropy_with_logits(input, target,mask, pos_weight=class_weights, reduction="sum")).numpy())

140291776

In [87]:
bcesum/torch.sum(mask)

tensor(0.)

In [144]:
loss_fn = torch.nn.BCEWithLogitsLoss(reduction="none", pos_weight=class_weights)

In [146]:
torch.sum(loss_fn(input, target)*mask)/torch.sum(mask)

tensor(3.7069)

In [208]:
class MaskedMultiLabelBCEwithLogits(torch.nn.BCEWithLogitsLoss):
    def __init__(self, pos_weight, spatial_dims=3):
        pos_weight = torch.Tensor(pos_weight)[(...,)+ (None,)*spatial_dims]
        self.loss_fn = super().__init__(reduction="none", pos_weight=pos_weight)
        self.spatial_dims = spatial_dims

    def forward(self, output, target, mask):
        bce = torch.sum(super().forward(output, target) * mask)
        bce /= torch.sum(mask)
        return bce


In [202]:
class_weights_list = list(class_weights.numpy().squeeze())

In [203]:
class_weights_list

[9.0, 11.0, 2.0, 13.0, 7.0, 1.0, 10.0, 5.0, 13.0, 11.0]

In [209]:
maloss = MaskedMultiLabelBCEwithLogits(class_weights_list)
maloss2 = MaskedMultiLabelBCEwithLogits([1,1,1,1,1,1,1,1,1,1])

torch.Size([10, 1, 1, 1])
torch.Size([10, 1, 1, 1])


In [210]:
maloss(input, target,mask)

tensor(3.7069)

In [211]:
maloss2(input, target, mask)

tensor(0.8060)

__main__.MaskedMultiLabelBCEwithLogits

In [165]:
maloss.loss_fn is None

True